In [1]:
import requests
from bs4 import BeautifulSoup
import re
import random
import numpy as np
import pandas as pd
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Bài 1

## 1.  Tiền xử lý dữ liệu với Beautiful Soup, re,...

In [2]:
def get_links(soup):
    items = soup.find_all('h3', class_='title-news')
    if len(items) == 0:
        items = soup.find_all('article', class_='item-news')
    links = [item.a['href'] for item in items if item.a is not None and item.a['href'].startswith('https://vnexpress.net/')]
    return links

def get_content(link):
    html = requests.get(link)
    sp = BeautifulSoup(html.text, 'html.parser')
    content = sp.find('div', class_='sidebar-1').find_all('p', class_=['Normal', 'description'], align='', style='')
    sentences = ''
    for i in content:
        if i.find('span') is not None:
            i.span.decompose()
        sentences += '\n'
        sentences += i.text
        sentences += ' '
    sentences = re.sub(r'\n+', ' ', sentences)
    sentences = re.sub(r'\s+', ' ', sentences)
    sentences = re.sub(r'[^\w\s.]', '', sentences)
    return sentences

def get_data(url):
    data = []
    i = 1
    while len(data) < 50:
        r = requests.get(url + '-p' + str(i))
        soup = BeautifulSoup(r.text, 'html.parser')
        links = get_links(soup)
        for link in links:
            sentences = get_content(link)
            if sentences != '':
                if len(data) >= 50:
                    break
                data.append(sentences)
        if len(data) < 50:
            i += 1
        else:
            i = 1
    return data

url = 'https://vnexpress.net/'

list_label = ['thoi-su', 'the-gioi', 'giai-tri', 'the-thao', 'khoa-hoc']

csv = []

for label in list_label:
    data = get_data(url + label)
    for i in data:
        csv.append([label, i])

random.shuffle(csv)

df = pd.DataFrame(csv, columns=['label', 'content'])
df.to_csv('data.csv', index=False)

## 2.  Tách từ (Tokenize) sử dụng thư viện pyvi hay underthesea

In [3]:
df = pd.read_csv('data.csv')
df.head()

,label,content
0,khoa-hoc,Lần đầu tiên một nhóm nhà khoa học ở Trung Qu...
1,giai-tri,Nhân vật ông Sắn Quang Tèo đóng liên tiếp gặp...
2,the-gioi,Hai trung đoàn hoàng gia sẽ bắn 103 phát đại ...
3,khoa-hoc,Khoảng 15.000 con gấu mèo sinh sống ở miền na...
4,khoa-hoc,Turbine gió ngoài khơi 16 MW do Trung Quốc tự...


In [4]:
from pyvi import ViTokenizer

def tokenize(text):
    return ViTokenizer.tokenize(text)

df_token = df.copy()

df_token['content'] = df_token['content'].apply(tokenize)
df_token.head()

,label,content
0,khoa-hoc,Lần đầu_tiên một nhóm nhà khoa_học ở Trung_Quố...
1,giai-tri,Nhân_vật ông Sắn Quang_Tèo đóng liên_tiếp gặp ...
2,the-gioi,Hai trung_đoàn hoàng_gia sẽ bắn 103 phát đại_b...
3,khoa-hoc,Khoảng 15.000 con gấu_mèo sinh_sống ở miền nam...
4,khoa-hoc,Turbine_gió ngoài khơi 16 MW do Trung_Quốc tự_...


## 3.  Trích xuất đặc trưng TF-IDF bằng thư viện sklearn

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df_token['content']).toarray()
y = df_token['label']

joblib.dump(vectorizer, 'vectorizer.pkl')

X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 4.  Đánh giá bộ dữ liệu với giải thuật KNN bằng phương pháp 5-Fold (k-fold)

In [6]:
knn = KNeighborsClassifier(n_neighbors=5)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

x = X

score = cross_val_score(knn,  x, y, cv=kf, scoring='accuracy').mean()

print('KNN: ', score)

KNN:  0.892


## 5, 6, 7, 8.  Huấn  luyện  dữ  liệu  cho  bài  toán  phân  loại văn bản  với  tỷ  lệ  dữ  liệu  8:2  (8  phần train, 2 phần test) sử dụng đặc trưng TF-IDF và 2 giải thuật Bayes, SVM. Đánh giá

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
bayer_model = MultinomialNB()
svm_model = SVC()

bayer_model.fit(x_train, y_train)
svm_model.fit(x_train, y_train)

y_pred_bayer = bayer_model.predict(x_test)
y_pred_svm = svm_model.predict(x_test)

print('Bayer:')
print('Accuracy: ', accuracy_score(y_test, y_pred_bayer))
print('F1 score: ', f1_score(y_test, y_pred_bayer, average='weighted'))
print('Confusion matrix: \n', confusion_matrix(y_test, y_pred_bayer))

print('\nSVM:')
print('Accuracy: ', accuracy_score(y_test, y_pred_svm))
print('F1 score: ', f1_score(y_test, y_pred_svm, average='weighted'))
print('Confusion matrix: \n', confusion_matrix(y_test, y_pred_svm))

max_score = accuracy_score(y_test, y_pred_bayer)
best_model = bayer_model

if accuracy_score(y_test, y_pred_svm) > max_score:
    max_score = accuracy_score(y_test, y_pred_svm)
    best_model = svm_model

joblib.dump(best_model, best_model.__class__.__name__ + '.pkl')

Bayer:
Accuracy:  0.84
F1 score:  0.8386363636363636
Confusion matrix: 
 [[ 8  0  1  5  0]
 [ 0 10  0  0  0]
 [ 0  0  7  0  1]
 [ 0  0  0  9  0]
 [ 0  0  0  1  8]]

SVM:
Accuracy:  0.88
F1 score:  0.8779893790420107
Confusion matrix: 
 [[11  1  0  1  1]
 [ 0 10  0  0  0]
 [ 0  1  6  0  1]
 [ 0  0  0  9  0]
 [ 1  0  0  0  8]]


['SVC.pkl']

Từ kết quả ta thấy SVC cho ra kết quả tốt hơn so với MultinomialNB với accuracy và f1-score là 0.92 và 0.92 so với 0.89 và 0.89.

In [9]:
svc_model = joblib.load('SVC.pkl')

def predict(text):
    text = tokenize(text)
    text = vectorizer.transform([text]).toarray()
    return svc_model.predict(text)[0]

predict('Tổng thống Mỹ Donald Trump đã ký sắc lệnh cấm TikTok và WeChat của Trung Quốc tại Mỹ, bắt đầu từ ngày 20/9.')

'the-gioi'

# Bài 2

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer 
corpus = [ 
     ' Hôm_nay tôi đi_học',     
     ' Hôm_nay tôi đi_học ở trường',     
     ' Hôm_nay tôi nghỉ ở nhà',    
     ' Hôm_nay tôi có đi_học không?', 
] 
vectorizer = HashingVectorizer(n_features=2**4) 
X = vectorizer.fit_transform(corpus) 
print(X.shape)

(4, 16)
